In [1]:
%cd /home/jovyan/notebooks/
from IPython.core.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Bangers" />
<style>
h1 {font-family: Bangers !important; font-size: 150px !important}
h2 {font-family: Bangers !important; font-size: 120px !important}
h3 {font-family: Bangers !important; font-size: 100px !important}
h4 {font-family: Bangers !important; font-size: 80px !important}
h5 {font-family: Bangers !important; font-size: 50px !important}
h6 {font-family: Bangers !important; font-size: 30px !important}
.txt7 {font-family: Bangers !important; font-size: 70px !important}
.txt6 {font-family: Bangers !important; font-size: 60px !important}
.txt5 {font-family: Bangers !important; font-size: 50px !important}
.txt4 {font-family: Bangers !important; font-size: 40px !important}
.txt3 {font-family: Bangers !important; font-size: 30px !important}
.cap {font-family: Bangers !important; font-size: 30px !important}
.controls
{
    display: none !important;
}

.slide-number
{
    display: none !important;
}
.inline-block {
   display: inline-block;
}
</style>
""")

/home/jovyan/notebooks


In [ ]:
!pip install yummy[polars] cleora==1.2.3 s3fs boto3 scikit-learn

#### Minio:

```bash
docker run --rm -it -p 9000:9000 \
 -p 9001:9001 --name minio \
 -v $(pwd)/minio-data:/data \
 --network app_default \
 minio/minio server /data --console-address ":9001"
```

In [2]:
import os 
import boto3
from botocore.client import Config
import numpy as np
import random

random.seed(0)
np.random.seed(0)

os.environ["AWS_ACCESS_KEY_ID"]= "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"]= "minioadmin"
os.environ["FEAST_S3_ENDPOINT_URL"]="http://minio:9000"
os.environ["S3_ENDPOINT_URL"]= "http://minio:9000"

s3 = boto3.resource('s3', endpoint_url='http://minio:9000')
s3.create_bucket(Bucket="input")
s3.create_bucket(Bucket="output")
s3.create_bucket(Bucket="data")

s3.Bucket(name='data')

#### Facebook Large Page-Page Network 

In [3]:
%cd /home/jovyan/notebooks
!curl -LO https://snap.stanford.edu/data/facebook_large.zip
!unzip facebook_large.zip

/home/jovyan/notebooks
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1696k  100 1696k    0     0   288k      0  0:00:05  0:00:05 --:--:--  376k
Archive:  facebook_large.zip
   creating: facebook_large/
  inflating: facebook_large/musae_facebook_edges.csv  
  inflating: facebook_large/musae_facebook_features.json  
  inflating: facebook_large/musae_facebook_target.csv  
  inflating: facebook_large/citing.txt  
  inflating: facebook_large/README.txt  


#### Prepare clique and star expansion input files

In [4]:
# based on: https://github.com/Synerise/cleora/blob/master/example_classification.ipynb
import pandas as pd
import s3fs
from sklearn.model_selection import train_test_split

df_cleora = pd.read_csv("./facebook_large/musae_facebook_edges.csv")
train_cleora, test_cleora = train_test_split(df_cleora, test_size=0.2)

fb_cleora_input_clique_filename = "s3://input/fb_cleora_input_clique.txt"
fb_cleora_input_star_filename = "s3://input/fb_cleora_input_star.txt"

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': "http://minio:9000"})

with fs.open(fb_cleora_input_clique_filename, "w") as f_cleora_clique, fs.open(fb_cleora_input_star_filename, "w") as f_cleora_star:
    grouped_train = train_cleora.groupby('id_1')
    for n, (name, group) in enumerate(grouped_train):
        group_list = group['id_2'].tolist()
        group_elems = list(map(str, group_list))
        f_cleora_clique.write("{} {}\n".format(name, ' '.join(group_elems)))
        f_cleora_star.write("{}\t{}\n".format(n, name))
        for elem in group_elems:
            f_cleora_star.write("{}\t{}\n".format(n, elem))

#### Cleora star expansion training

In [5]:
import time
import cleora
output_dir = 's3://output'
fb_cleora_input_star_filename = "s3://input/fb_cleora_input_star.txt"

start_time = time.time()
cleora.run(
    input=[fb_cleora_input_star_filename],
    type_name="tsv",
    dimension=1024,
    max_iter=5,
    seed=None,
    prepend_field=False,
    log_every=1000,
    in_memory_embedding_calculation=True,
    cols_str="transient::cluster_id StarNode",
    output_dir=output_dir,
    output_format="parquet",
    relation_name="emb",
    chunk_size=3000,
)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.242182493209839 seconds ---


[src/pipeline.rs:28] &sparse_matrices = [
    SparseMatrix {
        col_a_id: 0,
        col_a_name: "cluster_id",
        col_b_id: 1,
        col_b_name: "StarNode",
        edge_count: 0,
        hash_2_id: {},
        id_2_hash: [],
        row_sum: [],
        pair_index: {},
        entries: [],
    },
]


#### Cleora clique expansion training

In [6]:
fb_cleora_input_clique_filename = "s3://input/fb_cleora_input_clique.txt"
start_time = time.time()

cleora.run(
    input=[fb_cleora_input_clique_filename],
    type_name="tsv",
    dimension=1024,
    max_iter=5,
    seed=None,
    prepend_field=False,
    log_every=1000,
    in_memory_embedding_calculation=True,
    cols_str="complex::reflexive::CliqueNode",
    output_dir=output_dir,
    output_format="parquet",
    relation_name="emb",
    chunk_size=3000,
)
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.415393590927124 seconds ---


[src/pipeline.rs:28] &sparse_matrices = [
    SparseMatrix {
        col_a_id: 0,
        col_a_name: "CliqueNode",
        col_b_id: 1,
        col_b_name: "CliqueNode",
        edge_count: 0,
        hash_2_id: {},
        id_2_hash: [],
        row_sum: [],
        pair_index: {},
        entries: [],
    },
]


## Feast configuration

In [9]:
%cd /home/jovyan/notebooks/
!rm -r feature_store
!mkdir feature_store
%cd /home/jovyan/notebooks/feature_store

/home/jovyan/notebooks
/home/jovyan/notebooks/feature_store


In [10]:
%%writefile ./feature_store.yaml
project: repo
registry: s3://data/registry.db
provider: yummy.YummyProvider
backend: polars
online_store:
    type: sqlite
    path: data/online_store.db
offline_store:
    type: yummy.YummyOfflineStore

Writing ./feature_store.yaml


In [11]:
%%writefile ./features.py
from datetime import timedelta
from feast import Entity, Field, FeatureView
from yummy import ParquetSource
from feast.types import Float32, Int32

my_stats_parquet = ParquetSource(
    name="my_stats",
    path="s3://output/emb__CliqueNode__CliqueNode_*.parquet",
    timestamp_field="datetime",
    s3_endpoint_override="http://minio:9000",
)

my_entity = Entity(name="entity", description="entity",)

schema = [Field(name="entity", dtype=Int32)] + [Field(name=f"f{i}", dtype=Float32) for i in range(0,1024)]

mystats_view_parquet = FeatureView(
    name="my_statistics_parquet",
    entities=[my_entity],
    ttl=timedelta(seconds=3600*24*20),
    schema=schema,
    online=True, source=my_stats_parquet, tags={},)

Writing ./features.py


In [12]:
!feast apply

09/10/2022 08:42:31 PM botocore.credentials INFO: Found credentials in environment variables.
Created entity entity
Created feature view my_statistics_parquet

Deploying infrastructure for my_statistics_parquet


In [13]:
%cd /home/jovyan/notebooks/feature_store
from feast import FeatureStore
import polars as pl
import pandas as pd
import time
import os
from datetime import datetime
import yummy

store = FeatureStore(repo_path=".")
start_time = time.time()

features = [f"my_statistics_parquet:f{i}" for i in range(0,1024)]

training_df = store.get_historical_features(
    entity_df=yummy.select_all(datetime(2022, 9, 14, 23, 59, 42)),
    features = features,
).to_df()

print("--- %s seconds ---" % (time.time() - start_time))
training_df

/home/jovyan/notebooks/feature_store
I will use polars backend
--- 0.598198413848877 seconds ---


,entity,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f1015,f1016,f1017,f1018,f1019,f1020,f1021,f1022,f1023,event_timestamp
0,1,0.031914,-0.024654,0.016778,-0.018360,0.008336,-0.033789,-0.019724,-0.014500,-0.000028,...,-0.039011,-0.050917,0.053287,0.004578,0.056824,0.011253,0.014677,0.018925,0.009567,2022-09-14 23:59:42
1,16260,0.038295,-0.026572,0.011134,-0.018693,-0.027163,-0.001185,0.006900,-0.018135,0.014393,...,-0.045695,-0.068719,0.017536,0.007945,0.035621,0.006835,0.049093,0.039646,0.002602,2022-09-14 23:59:42
2,8049,0.024359,-0.023965,0.018983,-0.019353,0.005989,-0.023003,-0.022178,-0.012194,0.003375,...,-0.038033,-0.052741,0.050900,0.003405,0.059715,0.016017,0.017495,0.020577,0.009315,2022-09-14 23:59:42
3,9934,0.029962,-0.026767,0.019995,-0.017693,0.009001,-0.032112,-0.025161,-0.013655,0.000486,...,-0.038113,-0.051800,0.053348,0.001743,0.058919,0.012928,0.013532,0.015923,0.010026,2022-09-14 23:59:42
4,13737,0.028262,-0.023875,0.025852,-0.024644,0.006253,-0.034173,-0.017810,-0.017602,-0.001234,...,-0.036057,-0.052525,0.056144,-0.001642,0.062478,0.016809,0.013434,0.015423,0.005995,2022-09-14 23:59:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21838,22432,-0.028829,0.036718,0.055841,-0.013265,-0.041474,-0.038586,0.035573,0.020657,0.061384,...,-0.034968,-0.023898,-0.004210,-0.031653,0.061400,-0.024635,-0.031158,0.008509,-0.015728,2022-09-14 23:59:42
21839,22441,-0.008163,0.030377,-0.007110,0.044857,-0.048150,-0.052045,-0.021680,0.009681,0.035903,...,0.045597,0.014173,-0.031134,-0.022070,-0.050261,0.009182,0.016336,-0.060933,0.033865,2022-09-14 23:59:42
21840,21042,-0.041726,-0.016450,-0.021288,0.000353,0.025447,-0.018534,-0.018726,-0.091751,-0.029561,...,0.012627,0.013943,-0.038158,-0.011706,0.049925,0.068330,0.014162,-0.024143,0.025332,2022-09-14 23:59:42
21841,15349,-0.041726,-0.016450,-0.021288,0.000353,0.025447,-0.018534,-0.018726,-0.091751,-0.029561,...,0.012627,0.013943,-0.038158,-0.011706,0.049925,0.068330,0.014162,-0.024143,0.025332,2022-09-14 23:59:42


#### Prepare classificator training files

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv("../facebook_large/musae_facebook_target.csv")

classes = df['page_type'].unique()
class_ids = list(range(0, len(classes)))
class_dict = {k:v for k,v in zip(classes, class_ids)}
df['page_type'] = [class_dict[item] for item in df['page_type']]

train_filename = "fb_classification_train.txt"
test_filename = "fb_classification_test.txt"

train, test = train_test_split(df, test_size=0.2)

training_df=training_df.astype({"entity": "int32"})

entities = training_df["entity"].to_numpy()

train = train[["id","page_type"]].to_numpy()
test = test[["id","page_type"]].to_numpy()

df_embeddings=training_df.drop(columns=["event_timestamp"])\
    .rename(columns={ f"f{i}":i+2 for i in range(1024) })\
    .rename(columns={"entity": 0}).set_index(0)

valid_idx = df_embeddings.index.to_numpy()
train = np.array(train[np.isin(train[:,0], valid_idx) & np.isin(train[:,1], valid_idx)])
test = np.array([t for t in test if (t[0] in valid_idx) and (t[1] in valid_idx)])

## Traininig a classifier

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm
epochs=[20]
batch_size = 256
test_batch_size = 1000
embeddings=df_embeddings
y_train = train[:, 1]
y_test = test[:, 1]

clf = SGDClassifier(random_state=0, loss='log_loss', alpha=0.0001)
for e in tqdm(range(0, max(epochs))):
    for idx in range(0,train.shape[0],batch_size):
        ex=train[idx:min(idx+batch_size,train.shape[0]),:]
        ex_emb_in = embeddings.loc[ex[:,0]].to_numpy()
        ex_y = y_train[idx:min(idx+batch_size,train.shape[0])]
        clf.partial_fit(ex_emb_in, ex_y, classes=[0,1,2,3])
    
    if e+1 in epochs:
        acc = 0.0
        y_pred = []
        for n, idx in enumerate(range(0,test.shape[0],test_batch_size)):
            ex=test[idx:min(idx+test_batch_size,train.shape[0]),:]
            ex_emb_in = embeddings.loc[ex[:,0]].to_numpy()
            pred = clf.predict_proba(ex_emb_in)
            classes = np.argmax(pred, axis=1)
            y_pred.extend(classes)

        f1_micro = f1_score(y_test, y_pred, average='micro')
        f1_macro = f1_score(y_test, y_pred, average='macro')
        print(' epochs: {}, micro f1: {}, macro f1:{}'.format( e+1, f1_micro, f1_macro))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.19it/s]

 epochs: 20, micro f1: 0.9171151776103337, macro f1:0.9171222162436594
